### Fully connected tryout, input is set of points, it goes once through PointResnet which outputs 512 features, and once through PredictPlaneNet (fully connected prediction net for plane weights) which outputs K x 4 features, where K is number of kernels/planes used

In [1]:
## 3D Vision course implementation, most of it is being reused from the train.py file
## Author: Dusan

import torch
import torch.optim as optim
from tensorboardX import SummaryWriter
import numpy as np
import os
import argparse
import time
import matplotlib; matplotlib.use('Agg')
from im2mesh import config, data
from im2mesh.checkpoints import CheckpointIO
import torch.nn as nn
from im2mesh.encoder import fc_point_net


In [2]:
## mimicking for Shape3D and config.get_dataset() function 
## most of the setup has been copied from occupancy_networks/configs/default.yaml
## our config is in occupancy_networks/configs/point_plane_net.yaml
cfg = config.load_config('configs/point_plane_net_chair.yaml')


In [3]:
# Shorthands
out_dir = cfg['training']['out_dir']
batch_size = cfg['training']['batch_size']
# backup_every = cfg['training']['backup_every']
input_pc_size = cfg['data']['points_subsample']

In [4]:
# Dataset
train_dataset = config.get_dataset('train', cfg)
val_dataset = config.get_dataset('val', cfg)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, num_workers=4, shuffle=True,
    collate_fn=data.collate_remove_none,
    worker_init_fn=data.worker_init_fn)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=10, num_workers=4, shuffle=False,
    collate_fn=data.collate_remove_none,
    worker_init_fn=data.worker_init_fn)

In [5]:
L = 3

In [6]:
print(train_dataset[1000]['points'].shape)
print(train_dataset[1000]['points.occ'].shape)
print(train_dataset[1000]['inputs'].shape)
print(train_dataset[1000]['inputs.normals'].shape)

(1024, 3)
(1024,)
(256, 3)
(256, 3)


In [7]:
input_pc_size

1024

In [8]:
model = fc_point_net.FCModule(n_dim = 3, n_channels = L, n_points = input_pc_size)

In [9]:
input = torch.randn(10,input_pc_size, 3)
labels = torch.randn(10,512)

In [10]:
model(input).size()

torch.Size([10, 12])

In [11]:
model(input).view(10,3, -1)

tensor([[[ 0.3347, -0.0909, -0.2159,  0.1396],
         [-0.1090, -0.0856,  1.1209, -0.2445],
         [ 0.0199, -0.0127,  0.1947,  0.3696]],

        [[-0.2877, -0.1224, -0.3372, -0.2113],
         [ 0.3248,  0.0892,  0.1050,  0.1192],
         [ 0.1984,  0.0510,  0.2718,  0.1797]],

        [[-0.1797, -0.1329, -0.0260, -0.0439],
         [ 0.2757, -0.0734,  0.0430, -0.3174],
         [-0.2227,  0.5038,  0.1152,  0.0917]],

        [[-0.2383, -0.4676, -0.2296, -0.0074],
         [ 0.1063, -0.1447,  0.0056,  0.0880],
         [-0.3912,  0.3647,  0.4777,  0.2043]],

        [[-0.2047, -0.2737, -0.0381, -0.1202],
         [-0.3137, -0.0314,  0.7389, -0.5613],
         [-0.4740,  0.0540, -0.0263,  0.6056]],

        [[ 0.0776,  0.1875, -0.0183, -0.0247],
         [ 0.2516, -0.0885,  0.3075, -0.0517],
         [ 0.0156,  0.2411, -0.0370,  0.2593]],

        [[-0.6512,  0.0458,  0.0349, -0.2127],
         [ 0.2231, -0.0480,  0.3709,  0.1511],
         [ 0.0347,  0.4154,  0.2373,  0.5027]],
